In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [3]:
pathway = ''
#If you want, you can also pull the data from the mysql database.
df = pd.read_excel(pathway)
df.head()

,ifInOctets,ifOutOctets,ifInUcastPkts,ifInNUcastPkts,ifInDiscards,ifOutUcastPkts,ifOperStaus
0,5.006796e+09,6.028988e+09,8.151685e+07,208842.704602,0,2.424747e+07,1
1,8.490968e+04,2.434098e+04,9.652685e+01,0.097440,0,5.751417e+01,1
2,5.094763e+04,2.238871e+04,6.700774e+01,0.097448,0,4.989342e+01,1
3,7.640204e+04,3.176993e+04,8.514011e+01,0.097470,0,5.878664e+01,1
4,1.656709e+05,2.925770e+04,1.517357e+02,0.097454,0,7.270061e+01,1


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['ifInOctets', 'ifOutOctets', 'ifInUcastPkts', 'ifInNUcastPkts', 'ifInDiscards', 'ifOutUcastPkts', 'ifOperStaus']])

In [5]:
from tensorflow.keras import layers, models
input_dim = df_scaled.shape[1]

autoencoder = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')

In [6]:
history = autoencoder.fit(df_scaled, df_scaled, epochs=50, batch_size=32, validation_split=0.2, shuffle=True)

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.2425 - val_loss: 0.2128
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1920 - val_loss: 0.1020
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0704 - val_loss: 0.0097
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0067 - val_loss: 0.0014
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0014 - val_loss: 6.4459e-04
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0012 - val_loss: 4.1173e-04
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.6451e-04 - val_loss: 2.9149e-04
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.4794e-04 - val_loss: 2.1790e-04
Epoch 9/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0026 - val_loss: 1.6915e-04
Epoch 10/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0021 - val_loss: 1.3510e-04
Epoch 11/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6164e-04 - val_loss: 1.1037e-04
Epoch 12/50
26/26 ━━━━━━━━━━━━━━━

In [7]:

reconstructions = autoencoder.predict(df_scaled)
reconstruction_errors = tf.keras.losses.mse(reconstructions, df_scaled)

threshold = np.percentile(reconstruction_errors, 95)
anomalies = reconstruction_errors > threshold

df['is_anomaly'] = anomalies


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [8]:

sample_index = np.random.choice(df_scaled.shape[0], 1)
sample = df_scaled[sample_index].copy()

# Let's create an anomaly in the example  (excessively increase the ifInOctets value)
sample[:, 0] = sample[:, 0] * 100

print("Anomaly Data: \n", sample)



Anomaly Data: 
 [[1.03773399e+00 9.24079184e-07 2.71267731e-06 2.44291983e-09
  0.00000000e+00 9.64174488e-07 0.00000000e+00]]


In [9]:
reconstruction = autoencoder.predict(sample)
reconstruction_error = np.mean(np.square(sample - reconstruction))

print("reconstruction error:", reconstruction_error)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
reconstruction error: 0.15383964968144606


In [10]:
is_anomaly = reconstruction_error > threshold

print("Has an Anomaly been Detected?", is_anomaly)


Has an Anomaly been Detected? True
